In [3]:
import numpy as np
import pandas as pd
# stuff from probtest
CHECK_THRESHOLD = 1e-15
compute_statistics = ["mean", "max", "min"]

def parse_probtest_csv(path, index_col):
    df = pd.read_csv(path, index_col=index_col, header=[0, 1])

    times = df.columns.levels[0].astype(int)
    df.columns = df.columns.set_levels(times, level=0)

    # the dataframe's time column will be read as string,
    # thus ordered like "0", "1", "10", "11", .. "2", ...
    new_cols = pd.MultiIndex.from_product(
        [sorted(df.columns.levels[0]), df.columns.levels[1]],
        names=df.columns.names,
    )

    return pd.DataFrame(df, columns=new_cols)

def compute_rel_diff_dataframe(df1, df2):
    average = (df1 + df2) / 2
    out = (df1 - df2) / average
    out = out.abs()
    # put 0 if both numbers are very small
    zeros = np.logical_and(df1.abs() < CHECK_THRESHOLD, df2.abs() < CHECK_THRESHOLD)
    out[zeros] = 0.0
    return out

In [8]:
stats_cpu = parse_probtest_csv('../analysis/levante-qubic/stats_ref.csv', index_col=[0, 1, 2])
stats_gpu = parse_probtest_csv('../analysis/levante-qubic/stats_gpu_O2.csv', index_col=[0, 1, 2])

tend_ta_vdf_cpu = stats_cpu.loc[('NetCDF:*tend_ta_vdf_ml*.nc', 'tend_ta_vdf', slice(None)), :]
tend_ta_vdf_gpu = stats_gpu.loc[('NetCDF:*tend_ta_vdf_ml*.nc', 'tend_ta_vdf', slice(None)), :]
reldif = compute_rel_diff_dataframe(tend_ta_vdf_cpu, tend_ta_vdf_gpu)
reldif.abs().idxmax(axis=0)

/tmp/ipykernel_94279/2032966215.py:7: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  reldif.abs().idxmax(axis=0)


time  statistic
0     max           (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 4.0)
      mean         (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 11.0)
      min           (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 7.0)
1     max           (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 4.0)
      mean         (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 11.0)
      min           (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 7.0)
2     max           (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 6.0)
      mean         (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 11.0)
      min           (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 7.0)
3     max           (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 6.0)
      mean         (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 11.0)
      min           (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 3.0)
4     max           (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 7.0)
      mean         (NetCDF:*tend_ta_vdf_ml*.nc, tend_ta_vdf, 11.0)
      min           (NetCDF:*tend_ta_vdf_ml*.n

In [6]:
# mean is max at layer 11.0
v1_cpu = tend_ta_vdf_cpu.loc[('NetCDF:*tend_ta_vdf_ml*.nc', 'tend_ta_vdf', 11.0), (0, 'mean')]
v1_gpu = tend_ta_vdf_gpu.loc[('NetCDF:*tend_ta_vdf_ml*.nc', 'tend_ta_vdf', 11.0), (0, 'mean')]
print(v1_cpu, v1_gpu)
print(v1_cpu - v1_gpu, (v1_cpu - v1_gpu) / v1_gpu)

-2.2135070154213934e-14 -2.1800898518352112e-14
-3.3417163586182203e-16 0.015328342342427519
